## **Importing Libraries and Loading Files(Data)**

In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/pseudo-labels/pseudo_labels-face.npy
/kaggle/input/pseudo-labels/keys-speech.txt
/kaggle/input/pseudo-labels/keys-ocr.txt
/kaggle/input/pseudo-labels/pseudo_labels-audio.npy
/kaggle/input/pseudo-labels/pseudo_labels-rgb.npy
/kaggle/input/pseudo-labels/keys-rgb.txt
/kaggle/input/pseudo-labels/pseudo_labels-ocr.npy
/kaggle/input/pseudo-labels/pseudo_labels-object.npy
/kaggle/input/pseudo-labels/keys-face.txt
/kaggle/input/pseudo-labels/pseudo_labels-speech.npy
/kaggle/input/pseudo-labels/pseudo_labels-scene.npy
/kaggle/input/pseudo-labels/keys-audio.txt
/kaggle/input/pseudo-labels/keys-object.txt
/kaggle/input/pseudo-labels/keys-scene.txt
/kaggle/input/pseudo-labels-rgb/pseudo_labels.npy
/kaggle/input/pseudo-labels-rgb/keys.txt
/kaggle/input/msr-vtt-features-expert/structured-symlinks/val_list_jsfusion.txt
/kaggle/input/msr-vtt-features-expert/structured-symlinks/test_list_miech.txt
/kaggle/input/msr-vtt-features-expert/structured-symlinks/test_list_full.txt
/kaggle/input/m

In [3]:
!pip install zsvision

  Obtaining dependency information for zsvision from https://files.pythonhosted.org/packages/5b/64/ed80e275858e8e88fc8c46e73d9f15c8c660cd46c1e996d50c18453e3172/zsvision-0.7.12-py3-none-any.whl.metadata
  Obtaining dependency information for mergedeep from https://files.pythonhosted.org/packages/2c/19/04f9b178c2d8a15b076c8b5140708fa6ffc5601fb6f1e975537072df5b2a/mergedeep-1.3.4-py3-none-any.whl.metadata
  Obtaining dependency information for beartype>=0.3.2 from https://files.pythonhosted.org/packages/81/4a/97ea8a5afb289a25ae7db3b3ef68f0aad892bc1756be94565154877b173e/beartype-0.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for hickle>=4.0.0 from https://files.pythonhosted.org/packages/a0/fc/f91e9b1f6f5581b09c91339832a03b4e63444eb1a3565c9d5bad4abaebdd/hickle-5.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.4/872.4 kB 16.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 12.0 MB/s eta 0:00:00


In [4]:
import pickle
from zsvision.zs_utils import memcache, concat_features
!pip install einops
import torch
from torch import nn
import pdb
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import torch
from torch.utils.data import Dataset
import torchaudio
import random
import numpy as np
import os
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## **Dataloaders**

In [46]:
# initializing text embedding dim
text_dim = 768
class MSRVTTTrain(Dataset):
    def __init__(
        self,
        split="train",
        audio_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_audio_feats/Audio_MSRVTT_new.pickle',
        label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT_openAIGPT.pickle',
        rgb_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/senet154-avg.pickle',
        speech_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_speech/speech-w2v.pickle',
        ocr_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_ocr_feats/ocr-raw.pickle',
        scene_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_scene_25fps_256px_stride1_offset0/densenet161-avg.pickle',
        object_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/resnext101_32x48d-avg.pickle',
        face_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_face_feats/facefeats-avg.pickle',
        train_list = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/train_list_full.txt',
        pseudo_labels_path = "/kaggle/input/pseudo-labels-rgb/pseudo_labels.npy",
        keys_path = "/kaggle/input/pseudo-labels-rgb/keys.txt",
        num_position=16,   
        max_words = 30,
    ):
        
        self.num_position = num_position
        self.rgb_data = memcache(rgb_data_path)
        self.audio_data = memcache(audio_data_path)
        self.label = memcache(label_data_path)
        self.max_words = max_words
        self.speech_data = memcache(speech_data_path)
        self.ocr_data = memcache(ocr_data_path)
        self.scene_data = memcache(scene_data_path)
        self.face_data = memcache(face_data_path)
        self.object_data = memcache(object_data_path)
        
        #load pseudo labels
        #rgb
        self.rgb_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-rgb.npy")
        self.rgb_keys = []
        with open("/kaggle/input/pseudo-labels/keys-rgb.txt", "r") as f:
            for line in f:
                self.rgb_keys.append(line.strip())
        #audio
        self.audio_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-audio.npy")
        self.audio_keys = []
        with open("/kaggle/input/pseudo-labels/keys-audio.txt", "r") as f:
            for line in f:
                self.audio_keys.append(line.strip())
        #ocr
        self.ocr_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-ocr.npy")
        self.ocr_keys = []
        with open("/kaggle/input/pseudo-labels/keys-ocr.txt", "r") as f:
            for line in f:
                self.ocr_keys.append(line.strip())
        #speech
        self.speech_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-speech.npy")
        self.speech_keys = []
        with open("/kaggle/input/pseudo-labels/keys-speech.txt", "r") as f:
            for line in f:
                self.speech_keys.append(line.strip())
        #scene
        self.scene_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-scene.npy")
        self.scene_keys = []
        with open("/kaggle/input/pseudo-labels/keys-scene.txt", "r") as f:
            for line in f:
                self.scene_keys.append(line.strip())
        #face
        self.face_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-face.npy")
        self.face_keys = []
        with open("/kaggle/input/pseudo-labels/keys-face.txt", "r") as f:
            for line in f:
                self.face_keys.append(line.strip())
        #object
        self.object_pseudo = np.load("/kaggle/input/pseudo-labels/pseudo_labels-object.npy")
        self.object_keys = []
        with open("/kaggle/input/pseudo-labels/keys-object.txt", "r") as f:
            for line in f:
                self.object_keys.append(line.strip())
        #load train list and find splits
        self.split1 = []
        self.split2 = []
        
        with open(train_list) as f:
            self.train_list = f.readlines()
        self.train_list = [x.strip() for x in self.train_list]
        
        for key in self.train_list:
        # Check for modalities in the first split
            if isinstance(self.audio_data[key], float) == False and isinstance(self.ocr_data[key], float) == False and isinstance(self.speech_data[key], float) == False:
                self.split1.append(key)
            else:
                if isinstance(self.scene_data[key], float) == False :
                    self.split2.append(key)
        

    def __getitem__(self, index):
        key = self.train_list[index]
        #available_modalities = ["Object"]
        
        if key in self.split1:
            available_modalities = ["RGB", "Audio", "OCR", "Speech"]
        elif key in self.split2:
            available_modalities = ["RGB", "Face", "Scene", "Object"]
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        #pick random label each time
        r = random.randint(0, len(self.label[key])-1)
        label = self.label[key][r]
        
        #pick the first label
        #label = self.label[key][0]
        
        #------------Object------------
        output_data["Object"] = self.object_data[key]
        if "Object" in available_modalities:
            object_mask = np.ones((self.num_position, 1))
        else:
            object_mask = np.zeros((self.num_position, 1))
        index = self.object_keys.index(key)
        object_pseudo = torch.Tensor(self.object_pseudo[index])
        
        #------------Audio------------
        output_data["Audio"] = self.audio_data[key]
        if "Audio" in available_modalities:
            audio_mask = np.ones((self.num_position, 1))
        else:
            audio_mask = np.zeros((self.num_position, 1))
        index = self.audio_keys.index(key)
        audio_pseudo = torch.Tensor(self.audio_pseudo[index])
            
        #------------RGB------------
        output_data["RGB"] = self.rgb_data[key]
        if "RGB" in available_modalities:
            rgb_mask = np.ones((self.num_position, 1))
        else: 
            rgb_mask = np.zeros((self.num_position, 1))
        index = self.rgb_keys.index(key)
        rgb_pseudo = torch.Tensor(self.rgb_pseudo[index])
        
        
        #------------OCR------------
        output_data["OCR"] = self.ocr_data[key]
        if (isinstance(output_data["OCR"],float) != True) and "OCR" in available_modalities:
            ocr_mask = np.ones((self.num_position, 1))
        else:
            ocr_mask = np.zeros((self.num_position, 1))
        index = self.ocr_keys.index(key)
        ocr_pseudo = torch.Tensor(self.ocr_pseudo[index])
        
        #------------Speech------------
        output_data["Speech"] = self.speech_data[key]
        if (isinstance(output_data["Speech"],float) != True) and "Speech" in available_modalities:
            speech_mask = np.ones((self.num_position, 1))
        else:
            speech_mask = np.zeros((self.num_position, 1))
        index = self.speech_keys.index(key)
        speech_pseudo = torch.Tensor(self.speech_pseudo[index])
        
        #------------Face------------
        
        output_data["Face"] = self.face_data[key]
        if (isinstance(output_data["Face"],float) != True) and "Speech" in available_modalities:
            face_mask = np.ones((self.num_position, 1))
        else:
            face_mask = np.zeros((self.num_position, 1))
        index = self.face_keys.index(key)
        face_pseudo = torch.Tensor(self.face_pseudo[index])
        
        #------------Scene------------
        output_data["Scene"] = self.scene_data[key]
        if "Scene" in available_modalities:
            scene_mask = np.ones((self.num_position, 1))
        else:
            scene_mask = np.zeros((self.num_position, 1))
        index = self.scene_keys.index(key)
        scene_pseudo = torch.Tensor(self.scene_pseudo[index])
        
        masks = {
            "RGB": rgb_mask.astype(np.float32),
            "Audio": audio_mask.astype(np.float32),
            "OCR": ocr_mask.astype(np.float32),
            "Speech": speech_mask.astype(np.float32),
            "Face": face_mask.astype(np.float32),
            "Scene": scene_mask.astype(np.float32),
            "Object": object_mask.astype(np.float32),
        }

        
        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo,
            key
        )
    
    def collate_data(self, batch):
        keys = [item[-1] for item in batch]
        audio_batch = [item[0]["Audio"] for item in batch]
        rgb_batch = [item[0]["RGB"] for item in batch]
        face_batch = [item[0]["Face"] for item in batch]
        speech_batch = [item[0]["Speech"] for item in batch]
        ocr_batch = [item[0]["OCR"] for item in batch]
        scene_batch = [item[0]["Scene"] for item in batch]
        object_batch = [item[0]["Object"] for item in batch]
        
        label_batch = [item[1] for item in batch]
        
        rgb_masks_batch = [item[2]["RGB"] for item in batch]
        audio_masks_batch = [item[2]["Audio"] for item in batch]
        face_masks_batch = [item[2]["Face"] for item in batch]
        speech_masks_batch = [item[2]["Speech"] for item in batch]
        ocr_masks_batch = [item[2]["OCR"] for item in batch]
        scene_masks_batch = [item[2]["Scene"] for item in batch]
        object_masks_batch = [item[2]["Object"] for item in batch]
        
        audio_pseudo_batch = [item[3] for item in batch]
        rgb_pseudo_batch = [item[4] for item in batch]
        face_pseudo_batch = [item[5] for item in batch]
        speech_pseudo_batch = [item[6] for item in batch]
        ocr_pseudo_batch = [item[7] for item in batch]
        scene_pseudo_batch = [item[8] for item in batch]
        object_pseudo_batch = [item[9] for item in batch]
        
        video_tensor = np.zeros((len(rgb_batch), 2048))
        audio_tensor = np.zeros((len(audio_batch), self.max_words,128))
        text_tensor = np.zeros((len(label_batch), self.max_words, text_dim))
        face_tensor = np.zeros((len(face_batch), 512))
        speech_tensor = np.zeros((len(speech_batch), self.max_words, 300))
        ocr_tensor = np.zeros((len(ocr_batch), self.max_words, 300))
        scene_tensor = np.zeros((len(scene_batch), 1, 2208))
        object_tensor = np.zeros((len(object_batch),2048))
        
        for i in range(len(label_batch)):

            video_tensor[i] = rgb_batch[i] 
            
            object_tensor[i] = object_batch[i] 
            
            scene_tensor[i] = scene_batch[i]
            
            if (isinstance(face_batch[i], float) !=True):
                face_tensor[i] = face_batch[i]

            la = len(audio_batch[i])
            audio_tensor[i,:min(la,self.max_words), :] = audio_batch[i][:min(self.max_words,la)]

            lt = len(label_batch[i])
            text_tensor[i,:min(lt,self.max_words), :] = label_batch[i][:min(self.max_words,lt)]
            
            if (isinstance(ocr_batch[i], float) !=True):
                lo = len(ocr_batch[i])
                ocr_tensor[i,:min(lo,self.max_words), :] = ocr_batch[i][:min(self.max_words,lo)]

            if (isinstance(speech_batch[i], float) !=True):
                ls = len(speech_batch[i])
                speech_tensor[i,:min(ls,self.max_words), :] = speech_batch[i][:min(self.max_words,ls)]
                      
        masks = {
            "RGB": torch.stack([torch.from_numpy(mask).float() for mask in rgb_masks_batch]),
            "Audio": torch.stack([torch.from_numpy(mask).float() for mask in audio_masks_batch]),
            "OCR": torch.stack([torch.from_numpy(mask).float() for mask in ocr_masks_batch]),
            "Speech": torch.stack([torch.from_numpy(mask).float() for mask in speech_masks_batch]),
            "Face": torch.stack([torch.from_numpy(mask).float() for mask in face_masks_batch]),
            "Scene": torch.stack([torch.from_numpy(mask).float() for mask in scene_masks_batch]),
            "Object": torch.stack([torch.from_numpy(mask).float() for mask in object_masks_batch]),
        }
        
        audio_pseudo = torch.stack(audio_pseudo_batch)
        rgb_pseudo = torch.stack(rgb_pseudo_batch)
        speech_pseudo = torch.stack(speech_pseudo_batch)
        face_pseudo = torch.stack(face_pseudo_batch)
        ocr_pseudo = torch.stack(ocr_pseudo_batch)
        scene_pseudo = torch.stack(scene_pseudo_batch)
        object_pseudo = torch.stack(object_pseudo_batch)
        
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        output_data["Audio"] = torch.from_numpy(audio_tensor).float()
        output_data["RGB"] = torch.from_numpy(video_tensor).float()
        output_data["OCR"] = torch.from_numpy(ocr_tensor).float()
        output_data["Speech"] = torch.from_numpy(speech_tensor).float()
        output_data["Face"] = torch.from_numpy(face_tensor).float()
        output_data["Scene"] = torch.from_numpy(scene_tensor).float()
        output_data["Object"] = torch.from_numpy(object_tensor).float()
        
        label = torch.from_numpy(text_tensor).float()

        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo,
            keys
        )
    
    def __len__(self):
        return len(self.train_list)

In [16]:
class MSRVTTValidation(Dataset):
    def __init__(
        self,
        split="train",
        audio_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_audio_feats/Audio_MSRVTT_new.pickle',
        label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT_openAIGPT.pickle',
        rgb_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/senet154-avg.pickle',
        speech_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_speech/speech-w2v.pickle',
        ocr_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_ocr_feats/ocr-raw.pickle',
        scene_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_scene_25fps_256px_stride1_offset0/densenet161-avg.pickle',
        object_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/resnext101_32x48d-avg.pickle',
        face_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_face_feats/facefeats-avg.pickle',
        val_list = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/val_list_full.txt',
        num_position=16,   
        max_words = 30,
    ):
        
        self.num_position = num_position
        self.rgb_data = memcache(rgb_data_path)
        self.audio_data = memcache(audio_data_path)
        self.label = memcache(label_data_path)
        self.max_words = max_words
        self.speech_data = memcache(speech_data_path)
        self.ocr_data = memcache(ocr_data_path)
        self.scene_data = memcache(scene_data_path)
        self.face_data = memcache(face_data_path)
        self.object_data = memcache(object_data_path)
        
        #load validation list and find splits
        self.split = []
        
        with open(val_list) as f:
            self.val_list = f.readlines()
        self.val_list = [x.strip() for x in self.val_list]
        
        for key in self.val_list:
        # Check for modalities in the split
            if isinstance(self.audio_data[key], float) == False and isinstance(self.ocr_data[key], float) == False and isinstance(self.speech_data[key], float) == False and isinstance(self.scene_data[key], float) == False:
                self.split.append(key)

    def __getitem__(self, index):
        key = self.split[index]
        
        available_modalities = ["RGB", "Audio", "OCR", "Speech", "Face", "Scene", "Object"]
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        label = self.label[key][0]
        
        #------------Object------------
        output_data["Object"] = self.object_data[key]
        if "Object" in available_modalities:
            object_mask = np.ones((self.num_position, 1))
        else:
            object_mask = np.zeros((self.num_position, 1))
        object_pseudo = torch.zeros((text_dim,))
        
        #------------Audio------------
        output_data["Audio"] = self.audio_data[key]
        if "Audio" in available_modalities:
            audio_mask = np.ones((self.num_position, 1))
        else:
            audio_mask = np.zeros((self.num_position, 1))
        audio_pseudo = torch.zeros((text_dim,))
            
        #------------RGB------------
        output_data["RGB"] = self.rgb_data[key]
        if "RGB" in available_modalities:
            rgb_mask = np.ones((self.num_position, 1))
        else: 
            rgb_mask = np.zeros((self.num_position, 1))
        rgb_pseudo = torch.zeros((text_dim, ))
        
        #------------OCR------------
        output_data["OCR"] = self.ocr_data[key]
        if (isinstance(output_data["OCR"],float) != True) and "OCR" in available_modalities:
            ocr_mask = np.ones((self.num_position, 1))
        else:
            ocr_mask = np.zeros((self.num_position, 1))
        ocr_pseudo = torch.zeros((text_dim,))
        
        #------------Speech------------
        output_data["Speech"] = self.speech_data[key]
        if (isinstance(output_data["Speech"],float) != True) and "Speech" in available_modalities:
            speech_mask = np.ones((self.num_position, 1))
        else:
            speech_mask = np.zeros((self.num_position, 1))
        speech_pseudo = torch.zeros((text_dim,))
        
        #------------Face------------
        
        output_data["Face"] = self.face_data[key]
        if (isinstance(output_data["Face"],float) != True) and "Speech" in available_modalities:
            face_mask = np.ones((self.num_position, 1))
        else:
            face_mask = np.zeros((self.num_position, 1))
        face_pseudo = torch.zeros((text_dim,))
        
        #------------Scene------------
        output_data["Scene"] = self.scene_data[key]
        if "Scene" in available_modalities:
            scene_mask = np.ones((self.num_position, 1))
        else:
            scene_mask = np.zeros((self.num_position, 1))
        scene_pseudo = torch.zeros((text_dim,))
        
        masks = {
            "RGB": rgb_mask.astype(np.float32),
            "Audio": audio_mask.astype(np.float32),
            "OCR": ocr_mask.astype(np.float32),
            "Speech": speech_mask.astype(np.float32),
            "Face": face_mask.astype(np.float32),
            "Scene": scene_mask.astype(np.float32),
            "Object": object_mask.astype(np.float32),
        }

        
        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo
        )
    
    def collate_data(self, batch):
        audio_batch = [item[0]["Audio"] for item in batch]
        rgb_batch = [item[0]["RGB"] for item in batch]
        face_batch = [item[0]["Face"] for item in batch]
        speech_batch = [item[0]["Speech"] for item in batch]
        ocr_batch = [item[0]["OCR"] for item in batch]
        scene_batch = [item[0]["Scene"] for item in batch]
        object_batch = [item[0]["Object"] for item in batch]
        
        label_batch = [item[1] for item in batch]
        
        rgb_masks_batch = [item[2]["RGB"] for item in batch]
        audio_masks_batch = [item[2]["Audio"] for item in batch]
        face_masks_batch = [item[2]["Face"] for item in batch]
        speech_masks_batch = [item[2]["Speech"] for item in batch]
        ocr_masks_batch = [item[2]["OCR"] for item in batch]
        scene_masks_batch = [item[2]["Scene"] for item in batch]
        object_masks_batch = [item[2]["Object"] for item in batch]
        
        audio_pseudo_batch = [item[3] for item in batch]
        rgb_pseudo_batch = [item[4] for item in batch]
        face_pseudo_batch = [item[5] for item in batch]
        speech_pseudo_batch = [item[6] for item in batch]
        ocr_pseudo_batch = [item[7] for item in batch]
        scene_pseudo_batch = [item[8] for item in batch]
        object_pseudo_batch = [item[9] for item in batch]
        
        video_tensor = np.zeros((len(rgb_batch), 2048))
        audio_tensor = np.zeros((len(audio_batch), self.max_words,128))
        text_tensor = np.zeros((len(label_batch), self.max_words, text_dim))
        face_tensor = np.zeros((len(face_batch), 512))
        speech_tensor = np.zeros((len(speech_batch), self.max_words, 300))
        ocr_tensor = np.zeros((len(ocr_batch), self.max_words, 300))
        scene_tensor = np.zeros((len(scene_batch), 1, 2208))
        object_tensor = np.zeros((len(object_batch), 2048))
        
        for i in range(len(label_batch)):

            video_tensor[i] = rgb_batch[i] 
            
            object_tensor[i] = object_batch[i] 
            
            scene_tensor[i] = scene_batch[i]
            
            if (isinstance(face_batch[i], float) !=True):
                face_tensor[i] = face_batch[i]

            la = len(audio_batch[i])
            audio_tensor[i,:min(la,self.max_words), :] = audio_batch[i][:min(self.max_words,la)]

            lt = len(label_batch[i])
            text_tensor[i,:min(lt,self.max_words), :] = label_batch[i][:min(self.max_words,lt)]
            
            if (isinstance(ocr_batch[i], float) !=True):
                lo = len(ocr_batch[i])
                ocr_tensor[i,:min(lo,self.max_words), :] = ocr_batch[i][:min(self.max_words,lo)]

            if (isinstance(speech_batch[i], float) !=True):
                ls = len(speech_batch[i])
                speech_tensor[i,:min(ls,self.max_words), :] = speech_batch[i][:min(self.max_words,ls)]
                      
        masks = {
            "RGB": torch.stack([torch.from_numpy(mask).float() for mask in rgb_masks_batch]),
            "Audio": torch.stack([torch.from_numpy(mask).float() for mask in audio_masks_batch]),
            "OCR": torch.stack([torch.from_numpy(mask).float() for mask in ocr_masks_batch]),
            "Speech": torch.stack([torch.from_numpy(mask).float() for mask in speech_masks_batch]),
            "Face": torch.stack([torch.from_numpy(mask).float() for mask in face_masks_batch]),
            "Scene": torch.stack([torch.from_numpy(mask).float() for mask in scene_masks_batch]),
            "Object": torch.stack([torch.from_numpy(mask).float() for mask in object_masks_batch]),
        }
        
        audio_pseudo = torch.stack(audio_pseudo_batch)
        rgb_pseudo = torch.stack(rgb_pseudo_batch)
        speech_pseudo = torch.stack(speech_pseudo_batch)
        face_pseudo = torch.stack(face_pseudo_batch)
        ocr_pseudo = torch.stack(ocr_pseudo_batch)
        scene_pseudo = torch.stack(scene_pseudo_batch)
        object_pseudo = torch.stack(object_pseudo_batch)
        
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        output_data["Audio"] = torch.from_numpy(audio_tensor).float()
        output_data["RGB"] = torch.from_numpy(video_tensor).float()
        output_data["OCR"] = torch.from_numpy(ocr_tensor).float()
        output_data["Speech"] = torch.from_numpy(speech_tensor).float()
        output_data["Face"] = torch.from_numpy(face_tensor).float()
        output_data["Scene"] = torch.from_numpy(scene_tensor).float()
        output_data["Object"] = torch.from_numpy(object_tensor).float()
        
        label = torch.from_numpy(text_tensor).float()

        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo,
        )
    def __len__(self):
        return len(self.split)

In [17]:
class MSRVTTTEST(Dataset):
    def __init__(
        self,
        split="train",
        audio_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_audio_feats/Audio_MSRVTT_new.pickle',
        label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT_openAIGPT.pickle',
        rgb_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/senet154-avg.pickle',
        speech_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_speech/speech-w2v.pickle',
        ocr_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_ocr_feats/ocr-raw.pickle',
        scene_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_scene_25fps_256px_stride1_offset0/densenet161-avg.pickle',
        object_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_imagenet_25fps_256px_stride1_offset0/resnext101_32x48d-avg.pickle',
        face_data_path = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_face_feats/facefeats-avg.pickle',
        val_list = '/kaggle/input/msr-vtt-features-expert/structured-symlinks/test_list_full.txt',
        num_position=16,   
        max_words = 30,
    ):
        
        self.num_position = num_position
        self.rgb_data = memcache(rgb_data_path)
        self.audio_data = memcache(audio_data_path)
        self.label = memcache(label_data_path)
        self.max_words = max_words
        self.speech_data = memcache(speech_data_path)
        self.ocr_data = memcache(ocr_data_path)
        self.scene_data = memcache(scene_data_path)
        self.face_data = memcache(face_data_path)
        self.object_data = memcache(object_data_path)
        
        #load validation list and find splits
        self.split = []
        
        with open(val_list) as f:
            self.val_list = f.readlines()
        self.val_list = [x.strip() for x in self.val_list]
        
        for key in self.val_list:
        # Check for modalities in the split
            if isinstance(self.audio_data[key], float) == False and isinstance(self.ocr_data[key], float) == False and isinstance(self.speech_data[key], float) == False and isinstance(self.scene_data[key], float) == False:
                self.split.append(key)

    def __getitem__(self, index):
        key = self.split[index]
        
        available_modalities = ["RGB", "Audio", "OCR", "Speech", "Face", "Scene", "Object"]
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        label = self.label[key][0]
        
        #------------Object------------
        output_data["Object"] = self.object_data[key]
        if "Object" in available_modalities:
            object_mask = np.ones((self.num_position, 1))
        else:
            object_mask = np.zeros((self.num_position, 1))
        object_pseudo = torch.zeros((text_dim,))
        
        #------------Audio------------
        output_data["Audio"] = self.audio_data[key]
        if "Audio" in available_modalities:
            audio_mask = np.ones((self.num_position, 1))
        else:
            audio_mask = np.zeros((self.num_position, 1))
        audio_pseudo = torch.zeros((text_dim,))
            
        #------------RGB------------
        output_data["RGB"] = self.rgb_data[key]
        if "RGB" in available_modalities:
            rgb_mask = np.ones((self.num_position, 1))
        else: 
            rgb_mask = np.zeros((self.num_position, 1))
        rgb_pseudo = torch.zeros((text_dim, ))
        
        #------------OCR------------
        output_data["OCR"] = self.ocr_data[key]
        if (isinstance(output_data["OCR"],float) != True) and "OCR" in available_modalities:
            ocr_mask = np.ones((self.num_position, 1))
        else:
            ocr_mask = np.zeros((self.num_position, 1))
        ocr_pseudo = torch.zeros((text_dim,))
        
        #------------Speech------------
        output_data["Speech"] = self.speech_data[key]
        if (isinstance(output_data["Speech"],float) != True) and "Speech" in available_modalities:
            speech_mask = np.ones((self.num_position, 1))
        else:
            speech_mask = np.zeros((self.num_position, 1))
        speech_pseudo = torch.zeros((text_dim,))
        
        #------------Face------------
        
        output_data["Face"] = self.face_data[key]
        if (isinstance(output_data["Face"],float) != True) and "Speech" in available_modalities:
            face_mask = np.ones((self.num_position, 1))
        else:
            face_mask = np.zeros((self.num_position, 1))
        face_pseudo = torch.zeros((text_dim,))
        
        #------------Scene------------
        output_data["Scene"] = self.scene_data[key]
        if "Scene" in available_modalities:
            scene_mask = np.ones((self.num_position, 1))
        else:
            scene_mask = np.zeros((self.num_position, 1))
        scene_pseudo = torch.zeros((text_dim,))
        
        masks = {
            "RGB": rgb_mask.astype(np.float32),
            "Audio": audio_mask.astype(np.float32),
            "OCR": ocr_mask.astype(np.float32),
            "Speech": speech_mask.astype(np.float32),
            "Face": face_mask.astype(np.float32),
            "Scene": scene_mask.astype(np.float32),
            "Object": object_mask.astype(np.float32),
        }

        
        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo
        )
    
    def collate_data(self, batch):
        audio_batch = [item[0]["Audio"] for item in batch]
        rgb_batch = [item[0]["RGB"] for item in batch]
        face_batch = [item[0]["Face"] for item in batch]
        speech_batch = [item[0]["Speech"] for item in batch]
        ocr_batch = [item[0]["OCR"] for item in batch]
        scene_batch = [item[0]["Scene"] for item in batch]
        object_batch = [item[0]["Object"] for item in batch]
        
        label_batch = [item[1] for item in batch]
        
        rgb_masks_batch = [item[2]["RGB"] for item in batch]
        audio_masks_batch = [item[2]["Audio"] for item in batch]
        face_masks_batch = [item[2]["Face"] for item in batch]
        speech_masks_batch = [item[2]["Speech"] for item in batch]
        ocr_masks_batch = [item[2]["OCR"] for item in batch]
        scene_masks_batch = [item[2]["Scene"] for item in batch]
        object_masks_batch = [item[2]["Object"] for item in batch]
        
        audio_pseudo_batch = [item[3] for item in batch]
        rgb_pseudo_batch = [item[4] for item in batch]
        face_pseudo_batch = [item[5] for item in batch]
        speech_pseudo_batch = [item[6] for item in batch]
        ocr_pseudo_batch = [item[7] for item in batch]
        scene_pseudo_batch = [item[8] for item in batch]
        object_pseudo_batch = [item[9] for item in batch]
        
        video_tensor = np.zeros((len(rgb_batch), 2048))
        audio_tensor = np.zeros((len(audio_batch), self.max_words,128))
        text_tensor = np.zeros((len(label_batch), self.max_words, text_dim))
        face_tensor = np.zeros((len(face_batch), 512))
        speech_tensor = np.zeros((len(speech_batch), self.max_words, 300))
        ocr_tensor = np.zeros((len(ocr_batch), self.max_words, 300))
        scene_tensor = np.zeros((len(scene_batch), 1, 2208))
        object_tensor = np.zeros((len(object_batch), 2048))
        
        for i in range(len(label_batch)):

            video_tensor[i] = rgb_batch[i] 
            
            object_tensor[i] = object_batch[i] 
            
            scene_tensor[i] = scene_batch[i]
            
            if (isinstance(face_batch[i], float) !=True):
                face_tensor[i] = face_batch[i]

            la = len(audio_batch[i])
            audio_tensor[i,:min(la,self.max_words), :] = audio_batch[i][:min(self.max_words,la)]

            lt = len(label_batch[i])
            text_tensor[i,:min(lt,self.max_words), :] = label_batch[i][:min(self.max_words,lt)]
            
            if (isinstance(ocr_batch[i], float) !=True):
                lo = len(ocr_batch[i])
                ocr_tensor[i,:min(lo,self.max_words), :] = ocr_batch[i][:min(self.max_words,lo)]

            if (isinstance(speech_batch[i], float) !=True):
                ls = len(speech_batch[i])
                speech_tensor[i,:min(ls,self.max_words), :] = speech_batch[i][:min(self.max_words,ls)]
                      
        masks = {
            "RGB": torch.stack([torch.from_numpy(mask).float() for mask in rgb_masks_batch]),
            "Audio": torch.stack([torch.from_numpy(mask).float() for mask in audio_masks_batch]),
            "OCR": torch.stack([torch.from_numpy(mask).float() for mask in ocr_masks_batch]),
            "Speech": torch.stack([torch.from_numpy(mask).float() for mask in speech_masks_batch]),
            "Face": torch.stack([torch.from_numpy(mask).float() for mask in face_masks_batch]),
            "Scene": torch.stack([torch.from_numpy(mask).float() for mask in scene_masks_batch]),
            "Object": torch.stack([torch.from_numpy(mask).float() for mask in object_masks_batch]),
        }
        
        audio_pseudo = torch.stack(audio_pseudo_batch)
        rgb_pseudo = torch.stack(rgb_pseudo_batch)
        speech_pseudo = torch.stack(speech_pseudo_batch)
        face_pseudo = torch.stack(face_pseudo_batch)
        ocr_pseudo = torch.stack(ocr_pseudo_batch)
        scene_pseudo = torch.stack(scene_pseudo_batch)
        object_pseudo = torch.stack(object_pseudo_batch)
        
        output_data = {"Audio": None, "RGB": None, "OCR": None, "Speech": None, "Face": None, "Scene": None, "Object": None}
        output_data["Audio"] = torch.from_numpy(audio_tensor).float()
        output_data["RGB"] = torch.from_numpy(video_tensor).float()
        output_data["OCR"] = torch.from_numpy(ocr_tensor).float()
        output_data["Speech"] = torch.from_numpy(speech_tensor).float()
        output_data["Face"] = torch.from_numpy(face_tensor).float()
        output_data["Scene"] = torch.from_numpy(scene_tensor).float()
        output_data["Object"] = torch.from_numpy(object_tensor).float()
        
        label = torch.from_numpy(text_tensor).float()

        return (
            output_data,
            label,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo,
        )
    def __len__(self):
        return len(self.split)
        

## **Model Architecture**

In [19]:
##### Feature Reorganization ####
# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViTReorganization(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, pool = 'cls', dim_head = 64, dropout = 0., emb_dropout = 0., num_position = 16): 
        super().__init__()

        self.audio_embedding = nn.Sequential(
            nn.LayerNorm(128),
            nn.Linear(128, dim),
            nn.LayerNorm(dim)
        )
        self.rgb_embedding = nn.Sequential(
            nn.LayerNorm(2048),
            nn.Linear(2048, dim),
            nn.LayerNorm(dim)
        )
        self.face_embedding = nn.Sequential(
            nn.LayerNorm(512),
            nn.Linear(512, dim),
            nn.LayerNorm(dim)
        )
        
        self.speech_embedding = nn.Sequential(
            nn.LayerNorm(300),
            nn.Linear(300, dim),
            nn.LayerNorm(dim)
        )
        
        self.ocr_embedding = nn.Sequential(
            nn.LayerNorm(300),
            nn.Linear(300, dim),
            nn.LayerNorm(dim)
        )
        
        self.scene_embedding = nn.Sequential(
            nn.LayerNorm(2208),
            nn.Linear(2208, dim),
            nn.LayerNorm(dim)
        )
        
        self.object_embedding = nn.Sequential(
            nn.LayerNorm(2048),
            nn.Linear(2048, dim),
            nn.LayerNorm(dim)
        )

        self.embedding = {
            'Audio': self.audio_embedding,
            'RGB': self.rgb_embedding,
            'Face': self.face_embedding,
            'Speech': self.speech_embedding,
            'OCR': self.ocr_embedding,
            'Scene': self.scene_embedding,
            'Object': self.object_embedding,
        }

        self.rgb_pos_embedding = nn.Parameter(torch.randn(1,1, dim))
        self.audio_pos_embedding = nn.Parameter(torch.randn(1,30, dim))
        self.face_pos_embedding = nn.Parameter(torch.randn(1,1, dim))
        self.speech_pos_embedding = nn.Parameter(torch.randn(1,30, dim))
        self.ocr_pos_embedding = nn.Parameter(torch.randn(1,30, dim))
        self.scene_pos_embedding = nn.Parameter(torch.randn(1,1, dim))
        self.object_pos_embedding = nn.Parameter(torch.randn(1,1, dim))

        self.pos_embedding = {
            'Audio': self.audio_pos_embedding,
            'RGB': self.rgb_pos_embedding,
            'Face': self.face_pos_embedding,
            'Speech': self.speech_pos_embedding,
            'OCR': self.ocr_pos_embedding,
            'Scene': self.scene_pos_embedding,
            'Object': self.object_pos_embedding,
        }

        self.dropout = nn.Dropout(emb_dropout)

        self.rgb_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout) #mlp = multi layer perceptron
        self.audio_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.face_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.speech_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.ocr_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.scene_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.object_transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.transformer = {
            'RGB': self.rgb_transformer,
            'Audio': self.audio_transformer,
            'Face': self.face_transformer,
            'Speech': self.speech_transformer,
            'OCR': self.ocr_transformer,
            'Scene': self.scene_transformer,
            'Object': self.object_transformer,
        }

        self.pool = pool #where is pool used???
        self.to_latent = nn.Identity() #where is this used??

        self.rgb_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )

        self.audio_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.face_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.speech_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.ocr_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.scene_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.object_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_position)
        )
        
        self.heads = {
            'RGB': self.rgb_head,
            'Audio': self.audio_head,
            'Face': self.face_head,
            'Speech': self.speech_head,
            'OCR': self.ocr_head,
            'Scene': self.scene_head,
            'Object': self.object_head,
        }
        
        #fc_layer implemented by us
        ###
        self.rgb_fc_layer = nn.Linear(2048, dim)
        self.audio_fc_layer = nn.Linear(128, dim)
        self.face_fc_layer = nn.Linear(512, dim)
        self.speech_fc_layer = nn.Linear(300, dim)
        self.ocr_fc_layer = nn.Linear(300, dim)
        self.scene_fc_layer = nn.Linear(2208, dim)
        self.object_fc_layer = nn.Linear(2048, dim)
        ###
    
    def reorganization(self, feature, position):
        feature = torch.matmul(
            feature.transpose(1, 2).contiguous(), position
        )
        feature = feature.transpose(1, 2).contiguous()
        return feature

    def forward(self, rgb_inputs, audio_inputs, face_inputs, speech_inputs, ocr_inputs, scene_inputs, object_inputs): #B is batch size
        #unsqueeze rgb [B,2048] -> [B,1,2048] which is Fm in the paper
        rgb_inputs = rgb_inputs.unsqueeze(1)
        face_inputs = face_inputs.unsqueeze(1)
        object_inputs = object_inputs.unsqueeze(1)

        # rgb embeddings [B,1,2048] -> [B,1,256]
        rgb = self.rgb_embedding(rgb_inputs) + self.rgb_pos_embedding
        audio = self.audio_embedding(audio_inputs) + self.audio_pos_embedding
        face = self.face_embedding(face_inputs) + self.face_pos_embedding
        speech = self.speech_embedding(speech_inputs) + self.speech_pos_embedding
        ocr = self.ocr_embedding(ocr_inputs) + self.ocr_pos_embedding
        scene = self.scene_embedding(scene_inputs) + self.scene_pos_embedding
        objectt = self.object_embedding(object_inputs) + self.object_pos_embedding

        # rgb dropout [B,1,256] -> [B,1,256] some values go zero
        rgb = self.dropout(rgb)
        audio = self.dropout(audio)
        face = self.dropout(face)
        speech = self.dropout(speech)
        ocr = self.dropout(ocr)
        scene = self.dropout(scene)
        objectt = self.dropout(objectt)
        
        # rgb transformer [B,1,256] -> [B,1,256]
        rgb = self.rgb_transformer(rgb)
        audio = self.audio_transformer(audio)
        face = self.face_transformer(face)
        speech = self.speech_transformer(speech)
        ocr = self.ocr_transformer(ocr)
        scene = self.scene_transformer(scene)
        objectt = self.object_transformer(objectt)

        # rgb head [B,1,256] -> [B,1,16] which is Om in the paper
        rgb = self.rgb_head(rgb)
        audio = self.audio_head(audio)
        face = self.face_head(face)
        speech = self.speech_head(speech)
        ocr = self.ocr_head(ocr)
        scene = self.scene_head(scene)
        objectt = self.object_head(objectt)

        # rgb softmax [B,1,256] -> [B,1,16] which is Om normalized
        rgb = torch.softmax(rgb, dim = -1)
        audio = torch.softmax(audio, dim=-1)
        face = torch.softmax(face, dim=-1)
        speech = torch.softmax(speech, dim=-1)
        ocr = torch.softmax(ocr, dim=-1)
        scene = torch.softmax(scene, dim=-1)
        objectt = torch.softmax(objectt, dim=-1)
        
        # rgb reorganization [1,1,2048] transpose to [1,2048,1]// [1,2048,1]* [B,1,16] = [1,2048,16] transpose to [1,16,2048] which is F'm in the paper
        rgb = self.reorganization(rgb_inputs, rgb)
        audio = self.reorganization(audio_inputs, audio)
        face = self.reorganization(face_inputs, face)
        speech = self.reorganization(speech_inputs, speech)
        ocr = self.reorganization(ocr_inputs, ocr)
        scene = self.reorganization(scene_inputs, scene)
        objectt = self.reorganization(object_inputs, objectt)
        
        #rgb fully connected layer [1,16,2048] -> [1,16,256] which is F^m in the paper 
        #this part of code was missing from the researchers and was implemented by us 
        ###
        rgb = self.rgb_fc_layer(rgb)
        audio = self.audio_fc_layer(audio)
        face = self.face_fc_layer(face)
        speech = self.speech_fc_layer(speech)
        ocr = self.ocr_fc_layer(ocr)
        scene = self.scene_fc_layer(scene)
        objectt = self.object_fc_layer(objectt)
        
        
        #print(rgb.shape)
        #print(audio.shape)
        return rgb, audio, face, speech, ocr, scene, objectt

In [20]:
import torch
from torch import nn
import pdb
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import numpy as np

def MySoftmax(x, dim=-1, mask=None):
    maxes = torch.max(x*mask, dim, keepdim=True)[0]
    x_exp = torch.exp(x-maxes)

    if mask is not None:
        x_exp_sum = torch.sum(x_exp*mask, dim, keepdim=True)
        return x_exp * mask / (x_exp_sum + 1e-10)
    else:
        x_exp_sum = torch.sum(x_exp, dim, keepdim=True)
        return x_exp / x_exp_sum

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm_branch(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        # self.fn = fn
    def forward(self, x):
        return self.norm(x)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention_branch(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x, branch_num):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        qs = qkv[0]
        ks = qkv[1]
        vs = qkv[2]
        per_branch = int((qs.size()[1] - branch_num) // branch_num)
        outputs = []
        cls_outputs = []
        for i in range(branch_num):
            q = torch.cat((qs[:, i:(i+1), :], qs[:, (i*per_branch+branch_num) : ((i+1)*per_branch + branch_num), :] ), dim=1)
            q = rearrange(q, 'b n (h d) -> b h n d', h = self.heads)
            k = torch.cat((ks[:, i:(i+1), :], ks[:, (i*per_branch+branch_num) : ((i+1)*per_branch + branch_num), :] ), dim=1)
            k = rearrange(k, 'b n (h d) -> b h n d', h = self.heads)
            v = torch.cat((vs[:, i:(i+1), :], vs[:, (i*per_branch+branch_num) : ((i+1)*per_branch + branch_num), :] ), dim=1)
            v = rearrange(v, 'b n (h d) -> b h n d', h = self.heads)

            dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
            attn = self.attend(dots)
            attn = self.dropout(attn)
            out = torch.matmul(attn, v)
            out = rearrange(out, 'b h n d -> b n (h d)')
            cls_outputs.append(out[:,:1])
            outputs.append(out[:,1:])
        out = torch.cat(outputs, dim=1)
        cls_outputs = torch.cat(cls_outputs, dim=1)
        out = torch.cat((cls_outputs, out), dim=1)
        # q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        # dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        # attn = self.attend(dots)

        # device = x.get_device()
        # branch_masks = branch_masks.to(device)
        # attn_branches = 0
        # for i in range(branch_masks.size()[1]):
        #     attn = MySoftmax(dots, dim = -1, mask = branch_masks[:,i])#self.attend(dots)
        #     attn_branches += attn
        # attn = self.dropout(attn)

        # out = torch.matmul(attn, v)
        # out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer_branch(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm_branch(dim),
                Attention_branch(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                PreNorm_branch(dim),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))
    def forward(self, x, branch_num):
        for pn1, attn, pn2, ff in self.layers:
            x1 = pn1(x)
            x = attn(x1, branch_num) + x
            x1 = pn2(x)
            x = ff(x1) + x
        return x

class ViT(nn.Module):
    def __init__(self, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0., num_position = 16, branch_num = 2):
        super().__init__()
        self.mask_ratio = 0.5

        self.audio_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wronng
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )
        self.rgb_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )
        
        self.face_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )

        self.speech_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )

        self.ocr_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )
        
        self.scene_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )
        
        self.object_embedding = nn.Sequential(
            nn.LayerNorm(256), #might be wrong
            nn.Linear(256, dim),
            nn.LayerNorm(dim)
        )

        per_branch = num_position // branch_num
        branch_masks = []
        for i in range(branch_num):
            branch_mask1 = torch.zeros((1, 1, heads, num_position + branch_num, num_position + branch_num))
            branch_mask1[:, :, :, (i*per_branch + branch_num) :((i+1)*per_branch + branch_num), (i*per_branch + branch_num) :((i+1)*per_branch + branch_num)] = 1
            branch_mask1[:, :, :, i, (i*per_branch + branch_num) :((i+1)*per_branch + branch_num)] = 1 # cls mask
            branch_mask1[:, :, :, (i*per_branch + branch_num) :((i+1)*per_branch + branch_num), i] = 1 # cls mask
            branch_mask1[:,:,:,i,i] = 1
            branch_masks.append(branch_mask1)
        self.branch_masks = torch.cat(branch_masks, dim=1) #[1, branch_num, heads, num_position, num_position]
        self.branch_num = branch_num

        self.pos_embedding = nn.Parameter(torch.randn(1, num_position + branch_num, dim))
        self.cls_token = nn.Parameter(torch.randn(1, branch_num, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer_branch(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, rgb, audio, face, speech, ocr, scene, object_input, rgb_mask, audio_mask, face_mask, speech_mask, ocr_mask, scene_mask, object_mask):
        rgb = self.rgb_embedding(rgb)
        audio = self.audio_embedding(audio)
        face = self.face_embedding(face)
        speech = self.speech_embedding(speech)
        ocr = self.ocr_embedding(ocr)
        scene = self.scene_embedding(scene)
        objectt = self.object_embedding(object_input)
        
        features = rgb * rgb_mask + audio * audio_mask + face * face_mask + speech * speech_mask + ocr * ocr_mask + scene* scene_mask + objectt* object_mask
        
        b, n, _ = features.shape
        x = features + self.pos_embedding[:, self.branch_num:]

        cls_tokens = repeat(self.cls_token, '1 n d -> b n d', b = b)
        cls_tokens = cls_tokens + self.pos_embedding[:, :self.branch_num]
        x = torch.cat((cls_tokens, x), dim=1)
        
        x = self.transformer(x, self.branch_num)

        x = x[:, :self.branch_num]

        x = self.to_latent(x)
        x = self.mlp_head(x)

        return x


In [21]:

import pdb
import torch
import argparse
import tqdm
import os
import numpy as np
import torch.nn as nn
import random
import warnings
from torch.cuda.amp import GradScaler
import torch.nn.functional as F
import datetime

def dict_to_cuda(data):
    for key, value in data.items():
        data[key] = value.cuda()
    return data

def spatialtemporal2tokens(data):
    b, c, f, h, w = data.size()
    data = data.view(b, c, f * h * w)
    data = data.transpose(1, 2).contiguous()
    return data

class LabelSmoothLoss(nn.Module):
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * self.smoothing / (input.size(-1) - 1.0)
        weight.scatter_(-1, target.unsqueeze(-1), (1.0 - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

class AlignmentModule(nn.Module):
    def __init__(self, dim=256):
        super(AlignmentModule, self).__init__()
        self.base_vectors = nn.Parameter(torch.randn(1, 128, 256)) #changed from 3086 to 128
    
    def forward(self, input):
        # input [batchsize, n, 256]
        input = torch.mean(input, dim=1, keepdim=True) # [batch_size, 1, 256]
        #print("input",input.shape)
        base_vectors = self.base_vectors.repeat(input.size()[0], 1, 1)
        #print("base_vectors",base_vectors.shape)
        sim = torch.mean((base_vectors - input) ** 2, dim=-1)
        return sim

def extract_features(unimodal_models, data):
    outputs = {}
    for key, value in data.items():
        outputs[key] = unimodal_models[key](value)
        if key == 'RGB':
            outputs[key] = spatialtemporal2tokens(outputs[key])
    return outputs

In [22]:
class MaxMarginRankingLoss(nn.Module):
    def __init__(self, margin=1):
        super(MaxMarginRankingLoss, self).__init__()
        self.loss = torch.nn.MarginRankingLoss(margin)
        self.margin = margin

    def forward(self,x):
        n = x.size()[0]
        
        x1 = torch.diag(x)
        x1 = x1.unsqueeze(1)
        x1 = x1.expand(n, n)
        x1 = x1.contiguous().view(-1,1)
        x1 = torch.cat((x1,x1),0) 

        x2 = x.view(-1,1)
        x3 = x.transpose(0,1).contiguous().view(-1,1)
       
        x2 = torch.cat((x2,x3),0)
         
        max_margin = F.relu(self.margin - (x1 - x2))
        return max_margin.mean()

## **Initializing Everything**

In [49]:
torch.backends.cudnn.deterministic = True     #runs the same every time
torch.backends.cudnn.benchmark = False 
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)
warnings.filterwarnings("ignore")
device = "cuda"  # or 'cpu'
device = torch.device(device)
batch_size = 128
target_length = 128
multimodal_model = ViT(
        num_classes=text_dim,  #learnable tokens for the feature alignment loss(MSR_VTT= 128)
        dim=256, #d* -> which is the dim of the common space tokens
        depth=6, #number of transformer layers
        heads=8, 
        mlp_dim=512, #hidden dimension of feed-forward layer
        num_position=16 #k* -> number of tokens in common space
    )
multimodal_model = torch.nn.DataParallel(multimodal_model)
multimodal_model = multimodal_model.to(device)

    #feature projection model initialization
reorganization_module = ViTReorganization(
        dim=256,
        depth=6,
        heads=8,
        mlp_dim=512, 
)
reorganization_module = torch.nn.DataParallel(reorganization_module)
reorganization_module = reorganization_module.to(device)

alignment_model = AlignmentModule()
alignment_model = torch.nn.DataParallel(alignment_model)
alignment_model = alignment_model.to(device)

loss_fn = LabelSmoothLoss(smoothing=0.1)
loss_fn = loss_fn.cuda()

optim = torch.optim.SGD(
    list(multimodal_model.parameters())+list(reorganization_module.parameters()) + list(alignment_model.parameters()), lr=0.01, momentum=0.9, weight_decay=5e-4
)
### maybe change learning rate to lr=0.01

train_dataset = MSRVTTTrain(
        split="train",
        num_position=16,   # change to 16 for MSR-VTT
        #label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT.pickle'
    )

validation_dataset = MSRVTTValidation(
        num_position=16,   # change to 16 for MSR-VTT
        #label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT.pickle'
    )

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=train_dataset.collate_data,
        num_workers=4,
        pin_memory=True,
    )

val_loader = torch.utils.data.DataLoader(
        validation_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=validation_dataset.collate_data,
        num_workers=4,
        pin_memory=True,
    )

loss_fn = MaxMarginRankingLoss(margin=0.09381161988446174)

In [24]:
def verbose(epoch, status, metrics, name='TEST'):
    print(name + ' - epoch: %d, epoch status: %.2f, r@1: %.3f, r@5: %.3f, r@10: %.3f, mr: %d' %
          (epoch + 1, status,
           metrics['R1'], metrics['R5'], metrics['R10'],
           metrics['MR']))


def compute_metric(x):
    sx = np.sort(-x, axis=1)
    d = np.diag(-x)
    d = d[:, np.newaxis]
    ind = sx - d
    ind = np.where(ind == 0)
    ind = ind[1]

    metrics = {}
    metrics['R1'] = float(np.sum(ind == 0)) / len(ind)
    metrics['R5'] = float(np.sum(ind < 5)) / len(ind)
    metrics['R10'] = float(np.sum(ind < 10)) / len(ind)
    metrics['MR'] = np.median(ind) + 1

    return metrics


In [25]:
def verbose(epoch, mode, metrics, name="TEST"):
    r1, r5, r10, r50 = metrics["R1"], metrics["R5"], metrics["R10"], metrics["R50"]
    msg = f"{name:s} : {epoch}, R@1: {r1:.1f}"
    msg += f", R@5: {r5:.1f}, R@10 {r10:.1f}, R@50 {r50:.1f}"
    msg += f" MedR: {metrics['MedR']:g}, MeanR: {metrics['MeanR']:.1f}"
    print(msg)
    
def v2t_metrics(sims, query_masks=None):
    """Compute retrieval metrics from a similiarity matrix.

    Args:
        sims (th.Tensor): N x M matrix of similarities between embeddings, where
             x_{i,j} = <text_embd[i], vid_embed[j]>
        query_masks (th.Tensor): mask any missing captions from the dataset

    Returns:
        (dict[str:float]): retrieval metrics

    NOTES: We find the closest "GT caption" in the style of VSE, which corresponds
    to finding the rank of the closest relevant caption in embedding space:
    github.com/ryankiros/visual-semantic-embedding/blob/master/evaluation.py#L52-L56
    """
    # switch axes of text and video
    sims = sims.T

    if False:
        # experiment with toy example
        sims = np.ones((3, 3))
        sims[0, 0] = 2
        sims[1, 1:2] = 2
        sims[2, :] = 2
        query_masks = None

    assert sims.ndim == 2, "expected a matrix"
    num_queries, num_caps = sims.shape
    dists = -sims
    caps_per_video = num_caps // num_queries
    break_ties = "averaging"

    MISSING_VAL = 1E8
    query_ranks = []
    for ii in range(num_queries):
        row_dists = dists[ii, :]
        if query_masks is not None:
            # Set missing queries to have a distance of infinity.  A missing query
            # refers to a query position `n` for a video that had less than `n`
            # captions (for example, a few MSRVTT videos only have 19 queries)
            row_dists[np.logical_not(query_masks.reshape(-1))] = MISSING_VAL

        # NOTE: Using distance subtraction to perform the ranking is easier to make
        # deterministic than using argsort, which suffers from the issue of defining
        # "stability" for equal distances.  Example of distance subtraction code:
        # github.com/antoine77340/Mixture-of-Embedding-Experts/blob/master/train.py
        sorted_dists = np.sort(row_dists)

        min_rank = np.inf
        for jj in range(ii * caps_per_video, (ii + 1) * caps_per_video):
            if row_dists[jj] == MISSING_VAL:
                # skip rankings of missing captions
                continue
            ranks = np.where((sorted_dists - row_dists[jj]) == 0)[0]
            if break_ties == "optimistically":
                rank = ranks[0]
            elif break_ties == "averaging":
                # NOTE: If there is more than one caption per video, its possible for the
                # method to do "worse than chance" in the degenerate case when all
                # similarities are tied.  TODO(Samuel): Address this case.
                rank = ranks.mean()
            if rank < min_rank:
                min_rank = rank
        query_ranks.append(min_rank)
    query_ranks = np.array(query_ranks)

    # sanity check against old version of code
    if False:
        sorted_dists = np.sort(dists, axis=1)
        gt_dists_old = np.diag(dists)
        gt_dists_old = gt_dists_old[:, np.newaxis]
        rows_old, cols_old = np.where((sorted_dists - gt_dists_old) == 0)
        if rows_old.size > num_queries:
            _, idx = np.unique(rows_old, return_index=True)
            cols_old = cols_old[idx]
        num_diffs = (1 - (cols_old == query_ranks)).sum()
        msg = f"new metric doesn't match in {num_diffs} places"
        assert np.array_equal(cols_old, query_ranks), msg

        # visualise the distance matrix
        import sys
        import matplotlib
        matplotlib.use("Agg")
        import matplotlib.pyplot as plt
        sys.path.insert(0, str(Path.home() / "coding/src/zsvision/python"))
        from zsvision.zs_iterm import zs_dispFig # NOQA
        plt.matshow(dists)
        zs_dispFig()

    return cols2metrics(query_ranks, num_queries)

def cols2metrics(cols, num_queries):
    metrics = {}
    metrics["R1"] = 100 * float(np.sum(cols == 0)) / num_queries
    metrics["R5"] = 100 * float(np.sum(cols < 5)) / num_queries
    metrics["R10"] = 100 * float(np.sum(cols < 10)) / num_queries
    metrics["R50"] = 100 * float(np.sum(cols < 50)) / num_queries
    metrics["MedR"] = np.median(cols) + 1
    metrics["MeanR"] = np.mean(cols) + 1
    stats = [metrics[x] for x in ("R1", "R5", "R10")]
    return metrics

def t2v_metrics(sims, query_masks=None):
    """Compute retrieval metrics from a similiarity matrix.

    Args:
        sims (th.Tensor): N x M matrix of similarities between embeddings, where
             x_{i,j} = <text_embd[i], vid_embed[j]>
        query_masks (th.Tensor): mask any missing queries from the dataset (two videos
             in MSRVTT only have 19, rather than 20 captions)

    Returns:
        (dict[str:float]): retrieval metrics
    """
    assert sims.ndim == 2, "expected a matrix"
    num_queries, num_vids = sims.shape
    dists = -sims
    sorted_dists = np.sort(dists, axis=1)

    if False:
        import sys
        import matplotlib
        from pathlib import Path
        matplotlib.use("Agg")
        import matplotlib.pyplot as plt
        sys.path.insert(0, str(Path.home() / "coding/src/zsvision/python"))
        from zsvision.zs_iterm import zs_dispFig # NOQA
        plt.matshow(dists)
        zs_dispFig()
        import ipdb; ipdb.set_trace()

    # The indices are computed such that they slice out the ground truth distances
    # from the psuedo-rectangular dist matrix
    queries_per_video = num_queries // num_vids
    gt_idx = [[np.ravel_multi_index([ii, jj], (num_queries, num_vids))
              for ii in range(jj * queries_per_video, (jj + 1) * queries_per_video)]
              for jj in range(num_vids)]
    gt_idx = np.array(gt_idx)
    gt_dists = dists.reshape(-1)[gt_idx.reshape(-1)]
    gt_dists = gt_dists[:, np.newaxis]
    rows, cols = np.where((sorted_dists - gt_dists) == 0)  # find column position of GT

    # --------------------------------
    # NOTE: Breaking ties
    # --------------------------------
    # We sometimes need to break ties (in general, these should occur extremely rarely,
    # but there are pathological cases when they can distort the scores, such as when
    # the similarity matrix is all zeros). Previous implementations (e.g. the t2i
    # evaluation function used
    # here: https://github.com/niluthpol/multimodal_vtt/blob/master/evaluation.py and
    # here: https://github.com/linxd5/VSE_Pytorch/blob/master/evaluation.py#L87) generally
    # break ties "optimistically".  However, if the similarity matrix is constant this
    # can evaluate to a perfect ranking. A principled option is to average over all
    # possible partial orderings implied by the ties. See # this paper for a discussion:
    #    McSherry, Frank, and Marc Najork,
    #    "Computing information retrieval performance measures efficiently in the presence
    #    of tied scores." European conference on information retrieval. Springer, Berlin, 
    #    Heidelberg, 2008.
    # http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.145.8892&rep=rep1&type=pdf

    # break_ties = "optimistically"
    break_ties = "averaging"

    if rows.size > num_queries:
        assert np.unique(rows).size == num_queries, "issue in metric evaluation"
        if break_ties == "optimistically":
            _, idx = np.unique(rows, return_index=True)
            cols = cols[idx]
        elif break_ties == "averaging":
            # fast implementation, based on this code:
            # https://stackoverflow.com/a/49239335
            locs = np.argwhere((sorted_dists - gt_dists) == 0)

            # Find the split indices
            steps = np.diff(locs[:, 0])
            splits = np.nonzero(steps)[0] + 1
            splits = np.insert(splits, 0, 0)

            # Compute the result columns
            summed_cols = np.add.reduceat(locs[:, 1], splits)
            counts = np.diff(np.append(splits, locs.shape[0]))
            avg_cols = summed_cols / counts
            if False:
                print("Running slower code to verify rank averaging across ties")
                # slow, but more interpretable version, used for testing
                avg_cols_slow = [np.mean(cols[rows == idx]) for idx in range(num_queries)]
                assert np.array_equal(avg_cols, avg_cols_slow), "slow vs fast difference"
                print("passed num check")
            cols = avg_cols

    msg = "expected ranks to match queries ({} vs {}) "
    if cols.size != num_queries:
        import ipdb; ipdb.set_trace()
    assert cols.size == num_queries, msg

    if False:
        # overload mask to check that we can recover the scores for single-query
        # retrieval
        print("DEBUGGING MODE")
        query_masks = np.zeros_like(query_masks)
        query_masks[:, 0] = 1  # recover single query score

    if query_masks is not None:
        # remove invalid queries
        assert query_masks.size == num_queries, "invalid query mask shape"
        cols = cols[query_masks.reshape(-1).astype(np.bool)]
        assert cols.size == query_masks.sum(), "masking was not applied correctly"
        # update number of queries to account for those that were missing
        num_queries = query_masks.sum()

    if False:
        # sanity check against old logic for square matrices
        gt_dists_old = np.diag(dists)
        gt_dists_old = gt_dists_old[:, np.newaxis]
        _, cols_old = np.where((sorted_dists - gt_dists_old) == 0)
        assert np.array_equal(cols_old, cols), "new metric doesn't match"

    return cols2metrics(cols, num_queries)

## **Training**

In [ ]:
import time
epoch_outputs = []
BestLoss = float("inf")
BestEpoch = 0
print("---------------Start Training---------------")
for epoch_i in range(50):
    all_truth_outputs = [] 
    all_keys =[]
    start_time = time.time()
    dataloaders = {"train": train_loader, "val":val_loader}
    split = "train"
    scaler = GradScaler()
    count = 0
    count_val = 0
    total_loss = 0
    total_loss_val = 0
    loss = 0
    multimodal_model.train(split == "train")
    reorganization_module.train(split == "train")
    for (
        i,
        (
            data,
            labels,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo,
            key
        ),
    ) in enumerate(dataloaders["train"]):
        
        data = dict_to_cuda(data)
        masks = dict_to_cuda(masks)
        labels = labels
        audio_pseudo = audio_pseudo.cuda()
        rgb_pseudo = rgb_pseudo.cuda()
        face_pseudo = face_pseudo.cuda()
        speech_pseudo = speech_pseudo.cuda()
        ocr_pseudo = ocr_pseudo.cuda()
        scene_pseudo = scene_pseudo.cuda()
        outputs = data 
        
        rgb, audio, face, speech, ocr, scene, objectt = reorganization_module(
        outputs['RGB'], outputs['Audio'], outputs["Face"], outputs["Speech"], outputs["OCR"], outputs["Scene"], outputs["Object"]
        )
        
        audio_sim = alignment_model(audio)
        rgb_sim = alignment_model(rgb)
        face_sim = alignment_model(face)
        speech_sim = alignment_model(speech)
        ocr_sim = alignment_model(ocr)
        scene_sim = alignment_model(scene)
        object_sim = alignment_model(objectt)
        
        ###first column is for ground truth labels and secod column for pseudo labels
        outputs = multimodal_model(
            rgb, audio, face, speech, ocr, scene, objectt, masks['RGB'], masks['Audio'], masks['Face'], masks['Speech'], masks['OCR'], masks['Scene'], masks['Object']
        )
        ###calculate indices
        audio_indices = torch.sum(masks['Audio'].squeeze(-1), dim=-1) > 0
        rgb_indices = torch.sum(masks['RGB'].squeeze(-1), dim=-1) > 0
        face_indices = torch.sum(masks['Face'].squeeze(-1), dim=-1) > 0
        speech_indices = torch.sum(masks['Speech'].squeeze(-1), dim=-1) > 0
        ocr_indices = torch.sum(masks['OCR'].squeeze(-1), dim=-1) > 0
        scene_indices = torch.sum(masks['Scene'].squeeze(-1), dim=-1) > 0
        object_indices = torch.sum(masks['Object'].squeeze(-1), dim=-1) > 0

        ###calculate ground truth loss
        truth_outputs = outputs[:,0,:]
        detached_outputs = truth_outputs.detach().cpu()
        all_truth_outputs.append(detached_outputs)
        all_keys.extend(key)
        
        truth_labels = labels.mean(dim=1)
        truth_outputs = F.normalize(truth_outputs, p=2, dim=1)
        truth_labels = F.normalize(truth_labels, p=2, dim=1)
        truth_outputs = truth_outputs.to(device)
        truth_labels = truth_labels.to(device)
        similarity_matrix = torch.matmul(truth_outputs, truth_labels.t())
        loss_supervised = loss_fn(similarity_matrix)
        
        #check if modalities exist
        audio_sim = audio_sim[audio_indices]
        rgb_sim = rgb_sim[rgb_indices]
        face_sim = face_sim[face_indices]
        speech_sim = speech_sim[speech_indices]
        ocr_sim = ocr_sim[ocr_indices]
        scene_sim = scene_sim[scene_indices]
        object_sim = object_sim[object_indices]
        
        ###calculate alignment loss
        audio_sim, _ = torch.min(audio_sim, dim=-1)
        rgb_sim, _ = torch.min(rgb_sim, dim=-1)
        face_sim, _ = torch.min(face_sim, dim=-1)
        speech_sim, _ = torch.min(speech_sim, dim=-1)
        ocr_sim, _ = torch.min(ocr_sim, dim=-1)
        scene_sim, _ = torch.min(scene_sim, dim=-1)
        object_sim, _ = torch.min(object_sim, dim=-1)
        
        alignment_loss = (torch.sum(rgb_sim)+torch.sum(audio_sim)+torch.sum(face_sim)+torch.sum(speech_sim)+torch.sum(ocr_sim)+torch.sum(scene_sim)+torch.sum(object_sim))/torch.sum(rgb_indices)+(torch.sum(audio_indices)+ torch.sum(face_indices)+ torch.sum(speech_indices)+ torch.sum(ocr_indices)+ torch.sum(scene_indices)+ torch.sum(object_indices))

        confusion_matrix = similarity_matrix.data.cpu().float().numpy()
        metrics = compute_metric(confusion_matrix)
        #verbose(epoch_i,batch_size * float(i) / len(dataset),metrics, 'TRAIN')
        
        ###calculate pseudo loss
        pseudo_outputs = outputs[:,1,:]
        pseudo_outputs = F.normalize(pseudo_outputs, p=2, dim=1)
        
        rgb_pseudo = F.normalize(rgb_pseudo, p=2, dim=1)
        rgb_pseudo = rgb_pseudo.to(device)
        rgb_cos_sim = F.cosine_similarity(truth_labels, rgb_pseudo, dim=1)
        audio_pseudo = F.normalize(audio_pseudo, p=2, dim=1)
        audio_pseudo = audio_pseudo.to(device)
        ocr_pseudo = F.normalize(ocr_pseudo, p=2, dim=1)
        ocr_pseudo = ocr_pseudo.to(device)
        speech_pseudo = F.normalize(speech_pseudo, p=2, dim=1)
        speech_pseudo = speech_pseudo.to(device)
        face_pseudo = F.normalize(face_pseudo, p=2, dim=1)
        face_pseudo = face_pseudo.to(device)
        scene_pseudo = F.normalize(scene_pseudo, p=2, dim=1)
        scene_pseudo = scene_pseudo.to(device)
        object_pseudo = F.normalize(object_pseudo, p=2, dim=1)
        object_pseudo = object_pseudo.to(device)
        
        pseudo_labels = rgb_pseudo
        max_cos_sim = rgb_cos_sim
        
        pseudo_list = [audio_pseudo, ocr_pseudo, scene_pseudo, speech_pseudo, face_pseudo, object_pseudo]
        for pseudo in pseudo_list:
            cos_sim = F.cosine_similarity(truth_labels, pseudo, dim=1)
            truth_table = cos_sim>max_cos_sim
            pseudo_labels[truth_table] = pseudo[truth_table]
            max_cos_sim[truth_table] = cos_sim[truth_table]
        

        pseudo_outputs = pseudo_outputs.to(device)
        similarity_matrix = torch.matmul(pseudo_outputs, pseudo_labels.t())
        loss_pseudo = loss_fn(similarity_matrix)
        ###calculate total loss
        
        #loss = alignment_loss*0.001 + loss_supervised
        loss = alignment_loss*0.001 + loss_supervised + loss_pseudo
        #loss = loss_supervised + loss_pseudo
        #loss = loss_supervised
        
        optim.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        total_loss += loss.item() * batch_size
        #count+= batch_size
        count += outputs.size()[0]
    
    final_truth_outputs = torch.cat(all_truth_outputs, dim=0)
    final_truth_outputs_np = final_truth_outputs.numpy()
    
    if ((epoch_i+1)>49):   
        #epoch_outputs.append(final_truth_outputs_np)
        np.save("/kaggle/working/pseudo_labels-object.npy", final_truth_outputs_np)
        
        with open("/kaggle/working/keys-object.txt", "w") as f:
            for id in all_keys:
                f.write(f"{id}\n")
    all_truth_outputs.clear()
    all_keys.clear()
       
    for (
        i,
        (
            data,
            labels,
            masks,
            audio_pseudo,
            rgb_pseudo,
            face_pseudo,
            speech_pseudo,
            ocr_pseudo,
            scene_pseudo,
            object_pseudo
        ),
    ) in enumerate(dataloaders["val"]):
        data = dict_to_cuda(data)
        masks = dict_to_cuda(masks)
        labels = labels
        audio_pseudo = audio_pseudo.cuda()
        rgb_pseudo = rgb_pseudo.cuda()
        face_pseudo = face_pseudo.cuda()
        speech_pseudo = speech_pseudo.cuda()
        ocr_pseudo = ocr_pseudo.cuda()
        scene_pseudo = scene_pseudo.cuda()
         
        
        with torch.no_grad():
            outputs = data
            rgb, audio, face, speech, ocr, scene, objectt = reorganization_module(
            outputs['RGB'], outputs['Audio'], outputs["Face"], outputs["Speech"], outputs["OCR"], outputs["Scene"], outputs["Object"]
            )
            outputs = multimodal_model(
            rgb, audio, face, speech, ocr, scene, objectt, masks['RGB'], masks['Audio'], masks['Face'], masks['Speech'], masks['OCR'], masks['Scene'], masks['Object']
            )
            ###calculate ground truth loss
            truth_outputs = outputs[:,0,:]
            truth_labels = labels.mean(dim=1)
            truth_outputs = F.normalize(truth_outputs, p=2, dim=1)
            truth_labels = F.normalize(truth_labels, p=2, dim=1)
            truth_outputs = truth_outputs.to(device)
            truth_labels = truth_labels.to(device)
            similarity_matrix = torch.matmul(truth_outputs, truth_labels.t())
            loss_supervised = loss_fn(similarity_matrix)
            confusion_matrix = similarity_matrix.data.cpu().float().numpy()
            #metrics = compute_metric(confusion_matrix)
            metrics = v2t_metrics(confusion_matrix)
            verbose("Video to Text",batch_size * float(i) / len(validation_dataset),metrics, 'Validation')
            total_loss_val += loss_supervised.item() * batch_size
            #count+= batch_size
            count_val += outputs.size()[0]
            
    if (total_loss_val / float(count_val)) < BestLoss:
            BestLoss = total_loss_val / float(count_val)
            BestEpoch = epoch_i+1
            save = {
                    "epoch": epoch_i+1,
                    "model": multimodal_model.state_dict(),
                    "reorganization": reorganization_module.state_dict(),
                    "alignment": alignment_model.state_dict(),
                    "optimizer": optim.state_dict(),
                    "best_loss": BestLoss,
            }

            torch.save(
                  save,  "/kaggle/working/best_multimodal_openai_BestLoss_o.pt"
            )
    if (epoch_i>48):
            BestLoss = total_loss_val / float(count_val)
            BestEpoch = epoch_i+1
            save = {
                    "epoch": epoch_i+1,
                    "model": multimodal_model.state_dict(),
                    "reorganization": reorganization_module.state_dict(),
                    "alignment": alignment_model.state_dict(),
                    "optimizer": optim.state_dict(),
                    "best_loss": BestLoss,
            }

            torch.save(
                  save,  "/kaggle/working/best_multimodal_openai_epoch50_o.pt"
            )
             
    print("Epoch: %02d" % (epoch_i+1))
    #print("Current Training loss: ", loss.item())
    print("Average Training loss: ", total_loss/float(count))
    print("Average Validation loss: ", total_loss_val/float(count_val))
    end_time = time.time()
    dur = end_time-start_time
    print(f"Epoch Duration: {dur:.2f} seconds")
    print("---------------------------------------")
#stacked_outputs = np.stack(epoch_outputs)
#average_output = np.mean(stacked_outputs, axis=0)
#np.save("/kaggle/working/pseudo_labels.npy", average_output)

## **Testing**

In [ ]:
## Testing###
multimodal_model = ViT(
        num_classes=text_dim,  #learnable tokens for the feature alignment loss(MSR_VTT= 128)
        dim=256, #d* -> which is the dim of the common space tokens
        depth=6, #number of transformer layers
        heads=8, 
        mlp_dim=512, #hidden dimension of feed-forward layer
        num_position=16 #k* -> number of tokens in common space
    )
multimodal_model = torch.nn.DataParallel(multimodal_model)
multimodal_model = multimodal_model.to(device)

    #feature projection model initialization
reorganization_module = ViTReorganization(
        dim=256,
        depth=6,
        heads=8,
        mlp_dim=512, 
)
reorganization_module = torch.nn.DataParallel(reorganization_module)
reorganization_module = reorganization_module.to(device)

checkpoint = torch.load("/kaggle/working/best_multimodal_openai_epoch50.pt")
multimodal_model.load_state_dict(checkpoint['model'])
multimodal_model.eval()

reorganization_module.load_state_dict(checkpoint['reorganization'])
reorganization_module.eval()

test_dataset = MSRVTTTEST(
        num_position=16,   # change to 16 for MSR-VTT
        #label_data_path='/kaggle/input/msr-vtt-features-expert/structured-symlinks/aggregated_text_feats/w2v_MSRVTT.pickle'
    )

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=test_dataset.collate_data,
        num_workers=4,
        pin_memory=True,
        drop_last=False,
    )

print("---------------Start Testing ---------------")
print("Checkpoint from: " + str(checkpoint['epoch']) + " epoch  with best loss " + str(checkpoint['best_loss'])  )

aggregated_outputs = []
aggregated_labels = []

for (
      i,
      (
        data,
        labels,
        masks,
        audio_pseudo,
        rgb_pseudo,
        face_pseudo,
        speech_pseudo,
        ocr_pseudo,
        scene_pseudo,
        object_pseudo
     ),
    ) in enumerate(test_loader):
        data = dict_to_cuda(data)
        masks = dict_to_cuda(masks)
        labels = labels
        audio_pseudo = audio_pseudo.cuda()
        rgb_pseudo = rgb_pseudo.cuda()
        face_pseudo = face_pseudo.cuda()
        speech_pseudo = speech_pseudo.cuda()
        ocr_pseudo = ocr_pseudo.cuda()
        scene_pseudo = scene_pseudo.cuda()
        
        with torch.no_grad():
            outputs = data
            rgb, audio, face, speech, ocr, scene, objectt= reorganization_module(
            outputs['RGB'], outputs['Audio'], outputs["Face"], outputs["Speech"], outputs["OCR"], outputs["Scene"],  outputs["Object"]
            )
            outputs = multimodal_model(
            rgb, audio, face, speech, ocr, scene, objectt, masks['RGB'], masks['Audio'], masks['Face'], masks['Speech'], masks['OCR'], masks['Scene'],  masks['Object']
            )
            #prepare and add to list 
            
            #outputs = outputs.mean(dim=1).cpu()
            #truth_outputs = F.normalize(outputs, p=2, dim=1).cpu()
            
            truth_outputs = F.normalize(outputs[:,0,:], p=2, dim=1).cpu()
            truth_labels = F.normalize(labels.mean(dim=1), p=2, dim=1).cpu()
            aggregated_outputs.append(truth_outputs)
            aggregated_labels.append(truth_labels)
            
# Concatenate all collected outputs and labels
final_outputs = torch.cat(aggregated_outputs, dim=0)
final_labels = torch.cat(aggregated_labels, dim=0)
similarity_matrix = torch.matmul(final_outputs, final_labels.t())
confusion_matrix = similarity_matrix.numpy()
metrics = v2t_metrics(confusion_matrix)
verbose("Video to Text",batch_size * float(i) / len(validation_dataset),metrics, 'Test')
metrics = t2v_metrics(confusion_matrix)
verbose("Text to Video",batch_size * float(i) / len(validation_dataset),metrics, 'Test')

## **Draft Results**

**Model with Lsupervised**

Checkpoint from: 20 epoch  with best loss 0.8163572897122601

Test : Video to Text, R@1: **3.0**, R@5: **11.9**, R@10 **20.4** MedR: **51**, MeanR: **111.6**

Test : Text to Video, R@1: **2.7**, R@5: **10.6**, R@10 **19.0** MedR: **44**, MeanR: **97.1**

**Model with Lsupervised + Lalign**

Checkpoint from: 40 epoch  with best loss 0.8537058191975271

Test : Video to Text, R@1: 3.4, R@5: 11.6, R@10 18.0, R@50 49.3 MedR: 52, MeanR: 122.7

Test : Text to Video, R@1: 2.5, R@5: 8.8, R@10 15.6, R@50 42.6 MedR: 69, MeanR: 141.0

**Model with Lsupervised + Lsudo**

Checkpoint from: 20 epoch  with best loss 0.8174096054918184

Test : Video to Text, R@1: **2.2**, R@5: **12.0**, R@10 **19.9**, R@50 50.5 MedR: **50**, MeanR: **111.4**

Test : Text to Video, R@1: **3.4**, R@5: **11.0**, R@10 **20.5**, R@50 54.2 MedR: **44**, MeanR: **96.7**

**Model with Lsupervised and random labels**

Checkpoint from: 20 epoch  with best loss 0.8158261156457616

Test : Video to Text, R@1: 3.9, R@5: 12.7, R@10 20.4, R@50 54.6 MedR: 41, MeanR: **99.3**

Test : Text to Video, R@1: 2.6, R@5: 9.9, R@10 18.7, R@50 52.8 MedR: 46, MeanR: **99.6**

**Model with Lsupervised, OpenAI random labels**

Checkpoint from: 50 epoch  with best loss 0.8630691288024421

Test : Video to Text, R@1: 4.1, R@5: 13.7, R@10 21.8, R@50 52.5 MedR: 46, MeanR: **105.7**

Test : Text to Video, R@1: 3.4, R@5: 10.3, R@10 17.8, R@50 48.4 MedR: 53, MeanR: **112.0**

**Model with Lsupervised, OpenAI random labels, margin=0.1**

Checkpoint from: 50 epoch  with best loss 0.03814773109015517

Test : Video to Text, R@1: **5.5**, R@5: 18.3, R@10 28.1, R@50 64.1 MedR: **30**, MeanR: **71.3**

Test : Text to Video, R@1: **3.7**, R@5: 11.9, R@10 20.7, R@50 55.0 MedR: **42**, MeanR: **91.9**

**Model with Lsupervised + Lalignment, OpenAI random labels, margin=0.1**

Checkpoint from: 50 epoch  with best loss 0.03528064442431833

Test : Video to Text, R@1: 5.5, R@5: 17.9, R@10 28.4, R@50 64.4 MedR: 30, MeanR: **71.2**

Test : Text to Video, R@1: 3.7, R@5: 11.9, R@10 20.7, R@50 55.3 MedR: 43, MeanR: **91.7**

**Model with Lsupervised + Lpseudo + Lalignment, OpenAI random labels, margin=0.1**

Checkpoint from: 50 epoch  with best loss 0.033720782422643945

Test : Video to Text, R@1: **5.6**, R@5: 19.5, R@10 28.2, R@50 63.7 MedR: 31, MeanR: **70.8**

Test : Text to Video, R@1: **4.1**, R@5: 13.5, R@10 20.7, R@50 58.3 MedR: 36, MeanR: **84.7**

**Model with Lsupervised + Lpseudo + Lalignment, OpenAI random labels, margin=0.1 and OBJECT**

Checkpoint from: 50 epoch  with best loss 0.031087267117237482

Test : Video to Text, R@1: 7.2, R@5: 21.2, R@10 30.7, R@50 68.0 MedR: 25, MeanR: **61.2**

Test : Text to Video, R@1: 3.9, R@5: 14.4, R@10 25.1, R@50 59.7 MedR: 37, MeanR: **77.8**

## **Final Results**

**Model with Lsupervised**

Checkpoint from: 50 epoch  with best loss 0.03112133844630925

Test : Video to Text, R@1: 7.2, R@5: 21.2, R@10 30.8, R@50 68.0 MedR: 25, MeanR: **61.1**

Test : Text to Video, R@1: 3.8, R@5: 14.4, R@10 24.8, R@50 59.6 MedR: 37, MeanR: **78.0**



**Model with Lsupervised + Lalignment**

Checkpoint from: 50 epoch  with best loss 0.031087267117237482

Test : Video to Text, R@1: 7.2, R@5: 21.2, R@10 30.7, R@50 68.0 MedR: 25, MeanR: **61.2**

Test : Text to Video, R@1: 3.9, R@5: 14.4, R@10 25.1, R@50 59.7 MedR: 37, MeanR: **77.8**





**Model with Lsupervised + Lpseudo + Lalignment**

Checkpoint from: 50 epoch  with best loss 0.031087267117237482

Test : Video to Text, R@1: 7.2, R@5: 21.2, R@10 30.7, R@50 68.0 MedR: 25, MeanR: **61.2**

Test : Text to Video, R@1: 3.9, R@5: 14.4, R@10 25.1, R@50 59.7 MedR: 37, MeanR: **77.8**
